Test du MarkDown

Ceci est mon premier MarkDown

Test du MarkDown number 3


In [5]:
import yfinance as yf
import pyspark as ps
import pandas as pd
from pyspark.sql import SparkSession

In [3]:
aapl_data = yf.Ticker("AAPL")
historical_data = aapl_data.history(period = "1mo")
historical_data.to_csv("apple.csv")

In [4]:
spark = SparkSession.builder \
    .appName("CSV Operations") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

24/10/10 11:27:57 WARN Utils: Your hostname, MacBook-Air-de-Virginia.local resolves to a loopback address: 127.0.0.1; using 192.0.0.1 instead (on interface en0)
24/10/10 11:27:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/10 11:27:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/10/10 11:28:12 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
